<a href="https://colab.research.google.com/github/giorgosstath16/AIDL_01/blob/main/A_Pathfinding_Game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pygame

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 21.8 MB 1.2 MB/s 


In [2]:
import pygame
import math
from queue import PriorityQueue

pygame 2.1.2 (SDL 2.0.16, Python 3.7.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
#in order to run in a UNIX /Linux system and avoid 'pygame.error: No available video device'
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"

In order to run the game it is better to execute the .py file better locally,
for faster results.

In [5]:
# define window (size and caption)
WIDTH = 800
WIN = pygame.display.set_mode((WIDTH, WIDTH))
pygame.display.set_caption("A* Path Finding Algorithm")

In [6]:
# define colors
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 255, 0)
YELLOW = (255, 255, 0)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
PURPLE = (128, 0, 128)
ORANGE = (255, 165 ,0)
GREY = (128, 128, 128)
TURQUOISE = (64, 224, 208)

In [16]:
# define spots (nodes) in the grid, in order to recall later
# and keep track of them and its neighbours
class Spot:
  def __init__(self, row, col, width, total_rows):
    self.row = row
    self.col = col
    self.x = row * width # define coordinates 
    self.y = col * width
    self.color = WHITE # default color is white
    self.neighbors = []
    self.width = width
    self.total_rows = total_rows

  # return spot position on grid
  def get_pos(self):
    return self.row, self.col

  # define position from spot state/color
  def is_closed(self):
    return self.color == RED

  # define position from spot state/color
  def is_open(self):
    return self.color == GREEN

  # define position from spot state/color
  def is_barrier(self):
    return self.color == BLACK

  # define position from spot state/color
  def is_start(self):
    return self.color == ORANGE

  # define position from spot state/color
  def is_end(self):
    return self.color == TURQUOISE

  # reset color to default
  def reset(self):
    self.color = WHITE

  # change state from white to orange if spot is the start
  def make_start(self):
    self.color = ORANGE

  # change state from white to red if spot has been visited
  def make_closed(self):
    self.color = RED

  # change state from white to green if current position
  def make_open(self):
    self.color = GREEN

  # change state to black if spot is barrier
  def make_barrier(self):
    self.color = BLACK

  # change state to turqoise if spot is the end/target
  def make_end(self):
    self.color = TURQUOISE

  # create color trail
  def make_path(self):
    self.color = PURPLE

  # create a rectangle
  def draw(self, win):
    pygame.draw.rect(win, self.color, (self.x, self.y, self.width, self.width))

  # 
  def update_neighbors(self, grid):
    self.neighbors = []
    if self.row < self.total_rows - 1 and not grid[self.row + 1][self.col].is_barrier(): # DOWN
      self.neighbors.append(grid[self.row + 1][self.col])

    if self.row > 0 and not grid[self.row - 1][self.col].is_barrier(): # UP
      self.neighbors.append(grid[self.row - 1][self.col])

    if self.col < self.total_rows - 1 and not grid[self.row][self.col + 1].is_barrier(): # RIGHT
      self.neighbors.append(grid[self.row][self.col + 1])

    if self.col > 0 and not grid[self.row][self.col - 1].is_barrier(): # LEFT
      self.neighbors.append(grid[self.row][self.col - 1])

  # compare spots
  def __lt__(self, other):
    return False
  
    

In [17]:
# define heuristic function between 2 points/spots (Manhattan)
def h(p1, p2):
	x1, y1 = p1
	x2, y2 = p2
	return abs(x1 - x2) + abs(y1 - y2)

In [18]:
# generate the grid
def make_grid(rows, width):
	grid = []
	gap = width // rows
	for i in range(rows):
		grid.append([])
		for j in range(rows):
			spot = Spot(i, j, gap, rows)
			grid[i].append(spot)

	return grid

In [19]:
# draw the grid
def draw_grid(win, rows, width):
	gap = width // rows
	for i in range(rows):
		pygame.draw.line(win, GREY, (0, i * gap), (width, i * gap))
		for j in range(rows):
			pygame.draw.line(win, GREY, (j * gap, 0), (j * gap, width))

In [20]:
# draw spot w/ their colors
def draw(win, grid, rows, width):
	win.fill(WHITE)

	for row in grid:
		for spot in row:
			spot.draw(win)

	draw_grid(win, rows, width)
	pygame.display.update()

In [21]:
# figure which spot is clicked
def get_clicked_pos(pos, rows, width):
	gap = width // rows
	y, x = pos

	row = y // gap
	col = x // gap

	return row, col

In [22]:
# define main function
def main(win, width):
	ROWS = 50
	grid = make_grid(ROWS, width)

  # define variables to track start/end positions
	start = None
	end = None

	run = True
	while run:
    # draw grid praphics
		draw(win, grid, ROWS, width)
		for event in pygame.event.get():
			if event.type == pygame.QUIT:
				run = False

			if pygame.mouse.get_pressed()[0]: # if left mouse button pressed
				pos = pygame.mouse.get_pos()
				row, col = get_clicked_pos(pos, ROWS, width) # row & columns clicked
				spot = grid[row][col] 
				if not start and spot != end: # add start point
					start = spot
					start.make_start()

				elif not end and spot != start: # add end point
					end = spot
					end.make_end()

				elif spot != end and spot != start: # if start/end points defined, every click->barriers
					spot.make_barrier()

			elif pygame.mouse.get_pressed()[2]: # if left mouse button pressed
        # right mouse click for spot reset
				pos = pygame.mouse.get_pos()
				row, col = get_clicked_pos(pos, ROWS, width)
				spot = grid[row][col]
				spot.reset()
				if spot == start:
					start = None
				elif spot == end:
					end = None

      # A* algorithm control button
			if event.type == pygame.KEYDOWN:
				if event.key == pygame.K_SPACE and start and end:
					for row in grid:
						for spot in row:
							spot.update_neighbors(grid) 

					algorithm(lambda: draw(win, grid, ROWS, width), grid, start, end)

				if event.key == pygame.K_c:
					start = None
					end = None
					grid = make_grid(ROWS, width)

	pygame.quit()


In [23]:
# define search agent
def algorithm(draw, grid, start, end):
	count = 0
	open_set = PriorityQueue()
	open_set.put((0, count, start)) # add the start node
	came_from = {} # dictionary as a node history logbook
  # define the not start node functions to infinity until to be updated w/ score
	g_score = {spot: float("inf") for row in grid for spot in row} 
	g_score[start] = 0
	f_score = {spot: float("inf") for row in grid for spot in row}
	f_score[start] = h(start.get_pos(), end.get_pos())

	open_set_hash = {start} # keep track object in the priority queue

	while not open_set.empty():
		for event in pygame.event.get():
			if event.type == pygame.QUIT:
				pygame.quit()

		current = open_set.get()[2]
		open_set_hash.remove(current)

    # graphic visualization of search agent activity
		if current == end:
			reconstruct_path(came_from, end, draw)
			end.make_end()
			return True

		for neighbor in current.neighbors:
			temp_g_score = g_score[current] + 1

			if temp_g_score < g_score[neighbor]:
        # update values as a better path
				came_from[neighbor] = current
				g_score[neighbor] = temp_g_score
				f_score[neighbor] = temp_g_score + h(neighbor.get_pos(), end.get_pos())
				if neighbor not in open_set_hash:
					count += 1
					open_set.put((f_score[neighbor], count, neighbor))
					open_set_hash.add(neighbor)
					neighbor.make_open()

		draw()

		if current != start:
			current.make_closed()

	return False


In [24]:
def reconstruct_path(came_from, current, draw):
	while current in came_from:
		current = came_from[current]
		current.make_path()
		draw()


In [ ]:
main(WIN, WIDTH)